In [ ]:
def debug_features(model, batch):
    model.eval()
    with torch.no_grad():
        pc = batch['point_cloud'][:1].to(device)
        sp = batch['sample_points'][:1].to(device)
        
        # Check plane diversity
        plane_normals, plane_features = model.plane_predictor(pc)
        print("Plane normals:", plane_normals[0])
        print("Plane normal differences:", torch.cdist(plane_normals[0], plane_normals[0]))
        
        # Check encoder features
        features = model.encoder(pc)
        print("Encoder features - mean:", features.mean().item(), "std:", features.std().item())
        
        # Check decoder inputs
        pos_enc = model.decoder.pos_encoding(sp)
        print("Pos encoding - mean:", pos_enc.mean().item(), "std:", pos_enc.std().item())
        
        # Check final features before MLP
        # This requires modifying your decoder forward() to return intermediate features

# Usage:
sample_batch = next(iter(train_loader))
debug_features(model, sample_batch)

In [ ]:
def debug_decoder(model, batch):
    model.eval()
    with torch.no_grad():
        pc = batch['point_cloud'][:1].to(device)
        sp = batch['sample_points'][:1].to(device)
        
        # Get intermediate outputs
        point_features = model.encoder(pc)
        plane_normals, plane_features = model.plane_predictor(pc)
        
        # Check positional encoding
        pos_encoded = model.decoder.pos_encoding(sp)
        print("Pos encoded shape:", pos_encoded.shape)
        print("Pos encoded range:", pos_encoded.min().item(), "to", pos_encoded.max().item())
        
        # Check if decoder is getting reasonable features
        # You'll need to modify your decoder to return intermediate values
        print("\nCheck your decoder's fc_layers weights and biases:")
        for i, layer in enumerate(model.decoder.fc_layers):
            if hasattr(layer, 'weight'):
                print(f"Layer {i} weight mean: {layer.weight.mean().item():.4f}")
                print(f"Layer {i} bias: {layer.bias.data.item() if layer.bias.numel() == 1 else layer.bias.data.mean().item():.4f}")

debug_decoder(model, sample_batch)
print("Decoder output range:", occupancy.min().item(), "to", occupancy.max().item())
print(model.decoder.fc_layers[-1])  

In [ ]:
def debug_decoder_sampling(model, batch):
    model.eval()
    with torch.no_grad():
        pc = batch['point_cloud'][:1].to(device)
        sp = batch['sample_points'][:1, :10].to(device)  # Only 10 points for clarity
        
        # Get processed features (mimic your forward pass)
        point_features = model.encoder(pc)
        plane_normals, plane_features = model.plane_predictor(pc)
        
        # Get projected features (simplified)
        projected_features = []
        for i in range(3):
            proj_feat = model.planar_projection._project_to_plane(
                pc, point_features, plane_normals[:, i, :]
            )
            projected_features.append(proj_feat)
        projected_features = torch.stack(projected_features, dim=1)  # (B, L, C, H, W)
        
        print("Projected features shape:", projected_features.shape)
        
        # Test decoder sampling
        pos_encoded = model.decoder.pos_encoding(sp)
        print("Pos encoded shape:", pos_encoded.shape)
        
        # Sample features from planes
        sampled_features = []
        for i in range(3):
            plane_coords = model.decoder._project_points_to_plane(sp, plane_normals[:, i, :])
            print(f"Plane {i} coords range: {plane_coords.min().item():.3f} to {plane_coords.max().item():.3f}")
            
            sampled = F.grid_sample(
                projected_features[:, i], 
                plane_coords.unsqueeze(2),
                mode='bilinear', padding_mode='border', align_corners=True
            ).squeeze(-1).transpose(1, 2)
            print(f"Sampled features {i} - mean: {sampled.mean().item():.4f}, std: {sampled.std().item():.4f}")
            sampled_features.append(sampled)

debug_decoder_sampling(model, sample_batch)

In [ ]:
def debug_planar_features(model, batch):
    model.eval()
    with torch.no_grad():
        pc = batch['point_cloud'][:1].to(device)
        sp = batch['sample_points'][:1].to(device)
        
        # Get planar features
        point_features = model.encoder(pc)
        plane_normals, plane_features = model.plane_predictor(pc)
        
        # Check projected features
        combined_features = point_features.unsqueeze(1).expand(-1, 3, -1, -1)
        
        for i in range(3):
            proj_feat = model.planar_projection._project_to_plane(
                pc, combined_features[:, i, :, :], plane_normals[:, i, :]
            )
            print(f"Plane {i} features - mean: {proj_feat.mean().item():.4f}, std: {proj_feat.std().item():.4f}")
            print(f"Plane {i} non-zero ratio: {(proj_feat > 0).float().mean().item():.3f}")

debug_planar_features(model, sample_batch)

In [ ]:
def test_calibration(model, val_loader, device):
    model.eval()
    with torch.no_grad():
        batch = next(iter(val_loader))
        pc = batch['point_cloud'].to(device)
        sp = batch['sample_points'].to(device)
        true_occ = batch['occupancy'].to(device)
        
        pred_logits, _ = model(pc, sp, return_plane_params=True)
        pred_ratio = (torch.sigmoid(pred_logits) > 0.5).float().mean().item()
        true_ratio = (true_occ > 0.5).float().mean().item()
        
        print(f"Prediction ratio: {pred_ratio:.3f}")
        print(f"True ratio: {true_ratio:.3f}")
        print(f"Target: 0.02-0.05 range")
        
        return pred_ratio

# Test after each change
test_calibration(model, val_loader, device)

In [ ]:
# Add this after model creation, before trainer.train():
print("\n🧪 QUICK ARCHITECTURE TEST")
print("-" * 40)

model.eval()
with torch.no_grad():
    sample_batch = next(iter(train_loader))
    point_cloud = sample_batch['point_cloud'][:2].to(device)  # 2 samples only
    sample_points = sample_batch['sample_points'][:2].to(device)
    occupancy = sample_batch['occupancy'][:2].to(device)
    
    print("Testing forward pass...")
    output, planes = model(point_cloud, sample_points)
    
    pred_ratio = (output > 0.5).float().mean().item()
    true_ratio = (occupancy > 0.5).float().mean().item()
    
    print(f"✓ Output shape: {output.shape}")
    print(f"✓ Planes shape: {planes.shape}")
    print(f"✓ Prediction ratio: {pred_ratio:.1%} (was 34%)")
    print(f"✓ True occupancy: {true_ratio:.1%}")
    print(f"✓ Architecture test: {'PASS' if pred_ratio < 0.2 else 'NEEDS TUNING'}")